<a href="https://colab.research.google.com/github/wenko99/Standalone_DDL/blob/master/Lec07/Lec07_EMNIST_Model_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Grading EMNIST Models

<br/>

Google Drive의 **EMNIST** 폴더 안에 **emnist-models**라는 폴더를 새로 만드시고,

그 안에 Model의 정보가 담긴 **.h5, .json** 파일들을 업로드해주세요.

또, Github Repo의 Lec07 폴더에 업로드된 **emnist-custom.zip**을 받아서 압축을 푸시고, 나오는 **emnist-custom** 폴더를 폴더 그대로 **EMNIST** 안에 업로드해주세요.

<br/>

## Download and Preprocess Test Data

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
cd drive/My Drive//EMNIST

In [0]:
ls

### Custom Dataset

In [0]:
import skimage as im
import numpy as np

In [0]:
# Define some constants
ascii_idx = [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 97, 98, 100, 101, 102, 103, 104, 110, 113, 114, 116]
img_path_prefix = ['emnist-custom/emnist-custom-digit/', 'emnist-custom/emnist-custom-large/', 'emnist-custom/emnist-custom-small/']
img_path_postfix = ['.jpg', '_placeholder.jpg']

test_data_custom = []
test_labels_custom = []

for i in range(len(ascii_idx)):
    # Set image path
    if(ascii_idx[i] >= 48 and ascii_idx[i] <= 57):
        img_path = img_path_prefix[0]
    elif(ascii_idx[i] >= 65 and ascii_idx[i] <= 90):
        img_path = img_path_prefix[1]
    else:
        img_path = img_path_prefix[2]
    img_path = img_path + chr(ascii_idx[i]) + img_path_postfix[0]
    
    # Read image and append to test data / labels (actual handwriting. already made)
    test_data_custom.append(im.util.invert(im.color.rgb2gray(im.io.imread(img_path))))
    test_labels_custom.append(i)
    
test_data_custom = np.array(test_data_custom)
test_labels_custom = np.array(test_labels_custom)

###Predefined Dataset

In [0]:
import pandas as pd

In [0]:
# Read csv file
df = pd.read_csv('emnist-balanced-test.csv', header=None)

# Shuffle Dataset
np.random.seed(10)
df = df.reindex(np.random.permutation(df.index))

# Separate into Data / Label
test_data = df.iloc[:, 1:].values / 255
test_labels = df.iloc[:, 0].values

# Reshape Data
test_data = test_data.reshape((test_data.shape[0], 28, -1))
test_data = np.transpose(test_data, (0, 2, 1))

##Visualize Test Data

In [0]:
import matplotlib.pyplot as plt

In [0]:
def visualize(data, labels, idx=0):
    fig = plt.figure()
    for i in range(10):
        ax = fig.add_subplot(2, 5, i + 1)
        image = data[idx]
        ax.imshow(image)
        ax.set_title("label = {}".format(chr(ascii_idx[labels[idx]])))
        idx += 1
    plt.subplots_adjust(hspace = 0.5, wspace = 0.3)
    plt.show()

    print("The shape of the image is {}".format(image.shape))

print("CUSTOM DATASET")
visualize(test_data_custom, test_labels_custom)
print("PREDEFINED DATASET")
visualize(test_data, test_labels)

## Download Model Architecture / Parameters

In [0]:
import tensorflow as tf
import os

In [0]:
models = []
model_names = []

for filename in os.listdir('./emnist-models'):
    if filename.endswith('.json'):
        path = os.path.join('./emnist-models', filename)
        with open(path, 'r') as f:
            model_tmp = tf.keras.models.model_from_json(f.read())
        path = path[:-17]
        path = path + 'weights.h5'
        model_tmp.load_weights(path)
        models.append(model_tmp)
        model_names.append(path[16:-11])
        continue
    else:
        continue

In [0]:
model_names

##Test Accuracy

###Custom Dataset

In [0]:
# Reshape Data so that Conv2D is applicable
test_data_custom = test_data_custom.reshape((test_data_custom.shape[0], test_data_custom.shape[1], test_data_custom.shape[2], -1))

In [0]:
def test_custom(model):
    prediction_custom = np.argmax(model.predict(test_data_custom), 1)

    acc_custom = 0
    for i in range(len(test_data_custom)):
        if prediction_custom[i] == test_labels_custom[i]:
            acc_custom += 1
    acc_custom /= len(test_data_custom)

    print('Accuracy over Custom Test Set is {}%'.format(round(acc_custom, 5) * 100))
    
    return round(acc_custom, 5) * 100

    #test_data_custom = test_data_custom.reshape((test_data_custom.shape[0], test_data_custom.shape[1], -1))

    #visualize(test_data_custom, prediction_custom)

###Predefined Dataset

In [0]:
# Reshape Data so that Conv2D is applicable
test_data = test_data.reshape((test_data.shape[0], test_data.shape[1], test_data.shape[2], -1))

In [0]:
def test_predefined(model):
    prediction = np.argmax(model.predict(test_data), 1)

    acc = 0
    for i in range(len(test_data)):
        if prediction[i] == test_labels[i]:
            acc += 1
    acc /= len(test_data)

    print('Accuracy over Predefined Test Set is {}%'.format(round(acc, 5) * 100))
    
    return round(acc, 5) * 100

    #test_data = test_data.reshape((test_data.shape[0], test_data.shape[1], -1))

    #visualize(test_data, prediction)

In [0]:
res_custom = {}
res_predefined = {}

for i in range(len(models)):
    print('\nModel : {}\n'.format(model_names[i]))
    res_custom.update({model_names[i] : test_custom(models[i])})
    res_predefined.update({model_names[i] : test_predefined(models[i])})

In [0]:
import operator

res_custom = sorted(res_custom.items(), key=operator.itemgetter(1),reverse=True)
res_predefined = sorted(res_predefined.items(), key=operator.itemgetter(1), reverse=True)

print("LEADERBOARD : CUSTOM DATASET\n")
for i in range(len(res_custom)):
    print(res_custom[i])
print("\nLEADERBOARD : PREDEFINED DATASET\n")
for i in range(len(res_predefined)):
    print(res_predefined[i])